# Introduction
Skeletons are the serialisable form of the bill of materials and can be loaded into an assembly.

This section will introduce more advanced features of the bill of materials and then perform operations. The previously created skeleton will be loaded.

In [ ]:
import json
from pathlib import Path

with open(Path("example_Modifying_a_Skeleton_from_Settings.json"), "r") as f:
    skeleton = json.load(f)

## Working with the Bill of Materials

With the skeleton defined, it is very easy to load it into an assembly.

In [ ]:
from bom_analysis.bom import Assembly

blanket = Assembly()
blanket.from_dict(skeleton, ref="blanket")
blanket.plot_hierarchy()

_Note, the above warnings are an important demonstration of what is happening when the skeleton is being interpreted. The nodes class was told to use a pandas dataframe, but, in order to maintain consitancy between the input and output the code attempts to give nodes (a dataframe instance) an attribute of "class_str" - which pandas does not like._

As shown, the blanket is now an instance of assembly, it contains all the children which can be accessed like attributes

In [ ]:
print(blanket.manifold)
print(blanket.breeding_zone)

And all the non-InBuilt have been populated.

In [ ]:
print(blanket.manifold.params)

In [ ]:
blanket.manifold.nodes.loc["x_coordinate", 0] = 0.1
print(blanket.manifold.nodes)

The parameter class that was defined has integrated pint units.

In [ ]:
blanket.manifold.params.mass

And the unit type must be maintained when changing values

In [ ]:
from bom_analysis import Q_

blanket.manifold.params.mass = Q_(10, "g")
print(blanket.manifold.params)

The assembly structure can be operated on using this example calculation.

In [ ]:
class CountComponents:
    def __init__(self, assembly):
        count = 0
        for sub in assembly:
            count += 1
        assembly.params.number_of_components = Q_(count)


CountComponents(blanket)
blanket.params

Using the default classes should ensure that the BOM is always json serialisable. The BOM can be turned into a .json serialisable dictionary using the to_dict function.

## Useful Methods
### flatten() - Accessing All Components within an Assembly
The structure can be flattened so that each object below is returned in the same dictionary

In [ ]:
print(blanket.flatten().keys())

### lookup() and lookup_params() - Looking up Information within an Assembly
These functions can be used to lookup all items below in the hierarchy

In [ ]:
print(blanket.lookup("material"))
print(blanket.lookup_params("mass"))

### add_defaults() - Load Default Parameters
Large number of default parameters can easily be loaded into the bill of materials via add_defaults in the component/assembly.

In [ ]:
from bom_analysis import Q_

blanket.add_defaults({"breeding_zone": {"mass": Q_(500, "kg")}})
print(blanket.breeding_zone.params)

# Summary
The brief example has shown some of the ways to work with the BOM.
There are a number of functions within the bill of material which help access it.
Manually creating an assembly can be time consuming so BOM Analysis provides a number of classes which can create a skeleton that can be loaded into an Assembly via from_dict.